In [87]:
import janitor
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd. set_option('display.max_columns', None)
pd. set_option('display.max_rows', 20)
sns.set(rc={'figure.figsize':(15,10)})
import warnings
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join(''))
if module_path not in sys.path:
    sys.path.append(module_path+"/python_sql/")

In [88]:
from connect_sql import ConnectSQL
connect_obj = ConnectSQL()
df = connect_obj.query_db("""
select 
	l.customer_id,
	c.created registration_date,
	l.creation_date loan_start_date,
	l.id loan_id,t.scheduled_payment_date,t.status,t.payment_date ,
	count(*) over (partition by t.loan_id order by t.scheduled_payment_date ) trans_num,
	case when m.cat_integration_type_code in ('API','POSUI') then 'Offline' else 'Online' end channel
	from loan l 
	join customer c on c.id=l.customer_id 
	join transaction t on t.loan_id =l.id
	left join merchant m on m.id=l.merchant_id 
	where l.status in('HISTORICAL', 'LATE','OUTSTANDING','DEFAULTED') 
	and l.customer_id is not null; """)
df.to_csv("data/creditline_metric_raw_data_20220411.csv")

In [91]:
loans=pd.read_csv("data/creditline_metric_raw_data_20220411.csv",index_col=0)

In [92]:
loans

,customer_id,registration_date,loan_start_date,loan_id,scheduled_payment_date,status,payment_date,trans_num,channel
0,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-09-25 00:36:23.298513,PAID,2020-09-25 00:36:23.298513,1,Online
1,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-10-10 20:00:03.580116,PAID,2020-10-10 20:00:03.580116,2,Online
2,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-10-26 20:00:03.981579,PAID,2020-10-26 20:00:03.981579,3,Online
3,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-11-09 13:41:58.383552,PAID,2020-11-09 13:41:58.383552,4,Online
4,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-11-24 13:41:58.383000,PAID,2020-11-24 13:41:58.383000,5,Online
...,...,...,...,...,...,...,...,...,...
1233132,97566,2021-11-15 20:18:39.112373+00:00,2022-04-12 08:35:16.568262,640639,2022-04-12 08:37:14.952497,PAID,2022-04-12 08:37:14.952497,1,Online
1233133,97566,2021-11-15 20:18:39.112373+00:00,2022-04-12 08:35:16.568262,640639,2022-04-27 08:37:14.952497,NEXT,NaN,2,Online
1233134,97566,2021-11-15 20:18:39.112373+00:00,2022-04-12 08:35:16.568262,640639,2022-05-12 08:37:14.952497,PENDING,NaN,3,Online
1233135,97566,2021-11-15 20:18:39.112373+00:00,2022-04-12 08:35:16.568262,640639,2022-05-27 08:37:14.952497,PENDING,NaN,4,Online


# Filters

In [94]:
curr_date=pd.to_datetime(pd.datetime.today())

In [95]:
loans['payment_date']=loans.apply(lambda x: x.payment_date if pd.notnull(x.payment_date) else curr_date,axis=1)

In [96]:
loans['scheduled_payment_date']=pd.to_datetime(pd.to_datetime(loans['scheduled_payment_date']).dt.date)
loans['payment_date']=pd.to_datetime(pd.to_datetime(loans['payment_date']).dt.date)

In [97]:
loans=loans.groupby('loan_id').filter(lambda g: (g.scheduled_payment_date <curr_date).all())
# -pd.to_timedelta(1, unit='d')

In [98]:
loans=loans.groupby('loan_id').filter(lambda x: x['trans_num'].count()==5 )

In [99]:
loans=loans[loans['trans_num'] != 1]

In [100]:
loans

,customer_id,registration_date,loan_start_date,loan_id,scheduled_payment_date,status,payment_date,trans_num,channel
1,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-10-10,PAID,2020-10-10,2,Online
2,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-10-26,PAID,2020-10-26,3,Online
3,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-11-09,PAID,2020-11-09,4,Online
4,414,2020-09-21 17:40:46.805220+00:00,2020-09-25 00:19:37.047229,49,2020-11-24,PAID,2020-11-24,5,Online
6,414,2020-09-21 17:40:46.805220+00:00,2020-10-07 00:09:56.151454,85,2020-10-23,PAID,2020-10-23,2,Online
...,...,...,...,...,...,...,...,...,...
829826,228760,2022-02-11 23:43:12.587499+00:00,2022-02-11 23:55:40.599865,406803,2022-04-12,PAID,2022-02-11,5,Offline
829838,221371,2022-02-06 14:26:14.267551+00:00,2022-02-11 23:56:53.285627,406808,2022-02-26,PAID,2022-02-26,2,Offline
829839,221371,2022-02-06 14:26:14.267551+00:00,2022-02-11 23:56:53.285627,406808,2022-03-13,PAID,2022-03-13,3,Offline
829840,221371,2022-02-06 14:26:14.267551+00:00,2022-02-11 23:56:53.285627,406808,2022-03-28,PAID,2022-03-28,4,Offline


In [101]:
loans['shift_pay_date']=(loans.sort_values(by=['scheduled_payment_date'], ascending=True)
                       .groupby(['loan_id'])['payment_date'].shift(1))

In [102]:
loans['late_flag']=loans.apply(lambda x: 1 if (x.scheduled_payment_date < x.payment_date) &
                               ((x.scheduled_payment_date<x.shift_pay_date)| (~(pd.notnull(x.shift_pay_date)))) else 0,axis=1)

In [103]:
loans['cum_late_flag']=loans.groupby(['loan_id'])['late_flag'].cumsum()

# late pay considered

In [104]:
late_trans=loans.assign(
    flag_1 = np.where(((loans['trans_num']==2)&(loans['cum_late_flag']==1)),1,0),
    
    flag_2 = np.where(((loans['trans_num']==3)&(loans['cum_late_flag']==2)),1,0),
    
    flag_3 = np.where(((loans['trans_num']==4)&(loans['cum_late_flag']==3)),1,0),
    
    flag_4 = np.where(((loans['trans_num']==5)&(loans['cum_late_flag']==4)),1,0)
              ).groupby('loan_id').agg('sum').reset_index()

In [105]:
late_trans

,loan_id,customer_id,trans_num,late_flag,cum_late_flag,flag_1,flag_2,flag_3,flag_4
0,49,1656,14,0,0,0,0,0,0
1,85,1656,14,0,0,0,0,0,0
2,137,1656,14,0,0,0,0,0,0
3,165,3964,14,0,0,0,0,0,0
4,168,3964,14,1,4,1,0,0,0
...,...,...,...,...,...,...,...,...,...
165902,406793,270320,14,0,0,0,0,0,0
165903,406794,164484,14,1,4,1,0,0,0
165904,406800,860496,14,0,0,0,0,0,0
165905,406803,915040,14,0,0,0,0,0,0


In [106]:
late_trans=late_trans[['loan_id','flag_1','flag_2','flag_3','flag_4']]

In [107]:
late_trans.to_csv("data/fraud_bad_flags-20220331.csv")

In [108]:
late_trans=pd.read_csv("data/fraud_bad_flags-20220331.csv",index_col=0)

In [119]:
reg_date=loans.reset_index().drop_duplicates('loan_id')[['customer_id','registration_date','loan_start_date','loan_id','channel']]

In [120]:
reg_date=reg_date.rename(columns={'loan_start_date':'loan_creation_date'})

In [131]:
fraud_flag=pd.merge(reg_date,late_trans,on='loan_id',how='inner')

In [132]:
fraud_flag['registration_date']=pd.to_datetime(fraud_flag['registration_date']).dt.date
fraud_flag['loan_creation_date']=pd.to_datetime(fraud_flag['loan_creation_date']).dt.date

In [133]:
fraud_flag.to_csv("data/fraud_bad_flags_20220401_v3.csv",index = 0)